In [8]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [9]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import theano.tensor as T
from theano import function
import seaborn as sns
from theano.tensor.shared_randomstreams import RandomStreams

In [10]:
# Import My own library for factor testing
import factorFilterFunctions as ff
#from config import *

In [11]:
path = ff.data_path # path

In [12]:
# --------------------------------------- Calculate Forward Adjusted Price ---------------------------------- #

In [13]:
# the necessary files
filenameAdjustFactor='LZ_GPA_CMFTR_CUM_FACTOR.csv'
filenamePirce='LZ_GPA_QUOTE_TCLOSE.csv'

In [31]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)
    AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=pd.read_csv(path+filenamePirce,infer_datetime_format=True,parse_dates=[0],index_col=0)
    PriceToAdj=PriceToAdj[:]

    #Calculate
    AdjFacforward = AdjFacBackward/AdjFacBackward.max()
    adjustedPrice = (AdjFacforward*PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [32]:
calAdjustedPrice()

Own_Factor_AdjustedPriceForward-1d


In [16]:
# --------------------------------------- Calculate ILLQ Factor ---------------------------------- #

In [17]:
# 
filenameClose = 'LZ_GPA_QUOTE_TCLOSE.csv'
filenameOpen = 'LZ_GPA_QUOTE_TOPEN.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'

In [29]:
def calcILLQ():
    openPrice = pd.read_csv(path+filenameOpen,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)
    volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if openPrice.shape != closePrice.shape:
        print openPrice.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = np.abs((closePrice - openPrice)/openPrice)/volume
        newdf = newdf.rolling(min_periods=20,window=20,center=False).mean()
        newdf.index.name = 'Own_Factor_ILLQ-1d'
        newdf.to_csv(path+'Own_Factor_ILLQ-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [30]:
calcILLQ().tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300630.SZ,603178.SH,603833.SH,601228.SH,603041.SH,603303.SH,300635.SZ,300636.SZ,601200.SH,603385.SH
LZ_GPA_DERI_OWNILLIQ_20-d,,,,,,,,,,,,,,,,,,,,,
2017-03-29,6.707835e-08,4.625214e-08,NaN,1.061284e-07,1.116079e-07,5.830602e-08,NaN,NaN,9.765138e-07,2.355659e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-30,6.869572e-08,4.953425e-08,NaN,1.088760e-07,1.135406e-07,6.426049e-08,NaN,NaN,9.848811e-07,2.511127e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# --------------------------------------- Calculate FCAP Adjusted Turnover Volume---------------------------------- #

In [28]:
# REMINDER: Code in this section can be used to neutralize any new factor! Use this to check some size-affected factor!!!

In [21]:
filenameTURNOVER = 'LZ_GPA_QUOTE_TVOLUME.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [22]:
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [55]:
TURNOVER = pd.read_csv(path+filenameTURNOVER,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

TURNOVER= simpleNormalize(TURNOVER)

FCAP1 = simpleNormalize(FCAP1 )

datelist = FCAP1.index.tolist()

In [25]:
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [26]:
adjustedTurnOver = neutralizeFactor(TURNOVER , FCAP1, datelist)

In [27]:
adjustedTurnOver.index.name = 'Own_Factor_AdjustedTurnOver-1d'
adjustedTurnOver.to_csv(path+'Own_Factor_AdjustedTurnOver-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Calculate FCAP Adjusted PB ---------------------------------- #

In [47]:
filenamePB = 'LZ_GPA_VAL_PB.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
PB = pd.read_csv(path+filenamePB ,infer_datetime_format=True,parse_dates=[0],index_col=0)
PB = simpleNormalize(PB)

In [49]:
adjustedPB = neutralizeFactor(PB , FCAP1, datelist)
adjustedPB.index.name = 'Own_Factor_AdjustedPB-1d'
adjustedPB.to_csv(path+'Own_Factor_AdjustedPB-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [50]:
pb1 = pd.read_csv(path+'Own_Factor_AdjustedPB-1d.csv',infer_datetime_format=True,parse_dates=[0],index_col=0)

In [ ]:
# --------------------------------------- Calculate  x-days return volatility  ---------------------------------- #

In [38]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [43]:
def calReturnVol(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Volatility_%dd' % period
    newdf.to_csv(path+'Own_Factor_Volatility_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf 

In [44]:
calReturnVol(90).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300630.SZ,603178.SH,603833.SH,601228.SH,603041.SH,603303.SH,300635.SZ,300636.SZ,601200.SH,603385.SH
Own_Factor_Volatility_90d,,,,,,,,,,,,,,,,,,,,,
2017-03-29,0.011565,0.013257,0.012309,0.018094,0.016718,0.012425,0.011211,NaN,0.020133,0.018094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-30,0.011815,0.012965,0.012294,0.017998,0.017192,0.012551,0.011199,NaN,0.019801,0.018205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-31,0.011882,0.012984,0.012266,0.018072,0.017260,0.012620,0.011224,NaN,0.019882,0.018240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-05,0.011945,0.013001,0.012302,0.018105,0.017359,0.012683,0.011233,NaN,0.019941,0.018065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-06,0.012014,0.013074,0.012365,0.018178,0.017459,0.012733,0.011241,NaN,0.020056,0.018147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
